In [1]:
!python -m pip install python-dotenv --quiet
!python -m pip install gym stable-baselines3[extra] --quiet


In [2]:
# !python -m pip install awscli boto3 --quiet

In [3]:
%load_ext dotenv
%dotenv env

In [4]:
# !aws s3 sync reinforcement_learning_incentives s3://dissertation-data-dmiller/reinforcement_learning_incentives  

In [5]:
# ! if ![ -d "rl_ready_data" ]; then aws s3 cp s3://dissertation-data-dmiller/rl_ready_data ./rl_ready_data --recursive; fi

In [6]:
# %load rl_constant.py
LABEL = [
    "continue_work_session_30_minutes"
]

METADATA = [
    "user_id",
    "session_30_raw",
    "cum_platform_event_raw",
    "cum_platform_time_raw",
    "cum_session_time_raw",
    "global_events_user",
    "global_session_time",
    
    "year",
    "month",
    "day",
    "hour",
    "minute",
    "second"
]

OUT_FEATURE_COLUMNS = [
    "country_count", 
    "date_hour_sin", 
    "date_hour_cos",
    "date_minute_sin",
    "date_minute_cos",
    
    "session_30_count",
    "session_5_count",
    "cum_session_event_count",
    "delta_last_event",
    "cum_session_time",
    
    "expanding_click_average",
    "cum_platform_time",
    "cum_platform_events",
    "cum_projects",
    "average_event_time",
    
    "rolling_session_time",
    "rolling_session_events",
    "rolling_session_gap",
    "previous_session_time",
    "previous_session_events",
]

PREDICTION_COLS = [
    'seq_10',
]


GROUPBY_COLS = ['user_id']

RL_STAT_COLS = [
    'session_size',
    'sim_size',
    'session_minutes',
    'sim_minutes',
    'reward',
    'session_30_raw',
    'cum_platform_time_raw',
    'global_session_time',
]


In [7]:
# %load callback
import numpy as np
import pandas as pd
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.logger import TensorBoardOutputFormat
from datetime import datetime

class DistributionCallback(BaseCallback):
    
    @classmethod
    def tensorboard_setup(cls, log_dir, log_freq):
        cls._log_dir = log_dir
        cls._log_freq = log_freq

    def _on_training_start(self) -> None:
        output_formats = self.logger.output_formats
        self.tb_formatter = next(f for f in output_formats if isinstance(f, TensorBoardOutputFormat))
    
    def _on_step(self) -> bool:
        if self.n_calls % self._log_freq == 0:
            dist_list = self.training_env.env_method('dists')
            values_to_log = np.concatenate([d for d in dist_list if d.shape[0] > 0])

            values_df = pd.DataFrame(
                values_to_log, 
                columns=RL_STAT_COLS + ['ended', 'incentive_index', 'n_episodes'] + ['date_time']
            )
            
            dist_session_time = (values_df['session_minutes'] - values_df['reward']).mean()
            dist_session_end = (values_df['session_size'] - values_df['ended']).mean()
            
            dist_sim_time = (values_df['reward'] - values_df['sim_minutes']).mean()
            dist_sim_end = (values_df['ended'] - values_df['sim_size']).mean()


            dist_inc_session = (values_df['session_size'] - values_df['incentive_index']).mean()
            dist_inc_end = (values_df['ended'] - values_df['incentive_index']).mean()
            dist_inc_sim_index = (values_df['sim_size'] - values_df['incentive_index']).mean()

            n_call = self.n_calls // self._log_freq
            
            self.tb_formatter.writer.add_scalar('distance/session/max_reward::decrease', dist_session_time, n_call)
            self.tb_formatter.writer.add_scalar('distance/session/max_ended::decrease', dist_session_end, n_call)
           
            self.tb_formatter.writer.add_scalar('distance/session/sim_reward::increase', dist_sim_time, n_call)
            self.tb_formatter.writer.add_scalar('distance/session/sim_ended::increase', dist_sim_end, n_call)
            
            
            self.tb_formatter.writer.add_scalar('distance/incentive/max_incentive::decrease', dist_inc_session, n_call)
            self.tb_formatter.writer.add_scalar('distance/incentive/ended_incentive', dist_inc_end, n_call)
            self.tb_formatter.writer.add_scalar('distance/incentive/sim_inc_placement::decrease', dist_inc_sim_index, n_call)
            self.tb_formatter.writer.add_scalar('distance/ended_sim_size::increase', dist_inc_sim_index, n_call)
            
            self.tb_formatter.writer.flush()
            
            current_time = datetime.now().strftime("%Y_%m_%d-%H_%M_%S")
            values_df.to_csv(f'{self._log_dir}/dist_{n_call}_{current_time}.csv', index=False)
            
        return True

In [8]:
# %load environment
# %load environment
import gym
import numpy as np
# from rl_constant import RL_STAT_COLS
from scipy.stats import norm

import numpy as np
from scipy.stats import norm 
import gym
from datetime import datetime

class CitizenScienceEnv(gym.Env):
    
    metadata = {'render.modes': ['human']}
    
    def __init__(self, dataset, out_features, n_sequences):
        """
        trajectories: dictionary of user_id to their respective trajectories.
        n_sequences: number of sequences used for preprocessing.
        n_features: number of features used for preprocessing.
        """
        super(CitizenScienceEnv, self).__init__()
        self.dataset = dataset
        self.n_episodes = 0
        self.n_sequences = n_sequences
        self.current_session = None
        self.current_session_index = 0
        self.reward = 0
        self.metadata_container = []
        self.n_sequences = n_sequences
        self.out_features = out_features
        
        self.action_space = gym.spaces.Discrete(2)
        self.observation_space = gym.spaces.Box(low=-1, high=1, shape=(len(out_features), n_sequences + 1), dtype=np.float32)

    def reset(self):
        self.n_episodes += 1
        user_to_run, session_to_run = self.dataset.sample(1)[['user_id', 'session_30_raw']].values[0]
        self.current_session = self._get_events(user_to_run, session_to_run)
        self.metadata = self._metadata()
        self.current_session_index = 0
        self.reward = 0
        return self._state()

    def step(self, action):
        self._take_action(action)
        next_state, done, meta = self._calculate_next_state()
        
        if done:
            curent_session_index = min(self.current_session_index, self.current_session.shape[0] - 1)
            self.metadata['ended'] = self.current_session.iloc[curent_session_index]['cum_session_event_raw']
            self.metadata['reward'] = self.reward
            self.metadata['date_time'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            self.metadata_container.append(self.metadata.values)
            return next_state, float(self.reward), done, meta
        else:
            self.reward = self.current_session.iloc[self.current_session_index]['reward'] 
            self.current_session_index += 1        
        return next_state, float(self.reward), done, meta
    
    def _metadata(self):
        session_metadata = self.current_session.iloc[0][RL_STAT_COLS]
        session_metadata['ended'] = 0
        session_metadata['incentive_index'] = 0
        session_metadata['n_episodes'] = self.n_episodes
        return session_metadata
    
    
    def _calculate_next_state(self):
        
        if (self.current_session_index == self.current_session.shape[0]):
            return None, True, {}

        if self._continuing_in_session():
            return self._state(), False, {}
    
        return None, True, {}
        
      
  
    def _continuing_in_session(self):
        sim_counts = self.metadata['sim_size']
        current_session_count = self.current_session.iloc[self.current_session_index]['cum_session_event_raw']
        if current_session_count < sim_counts:
            return True
        
        extending_session = self._probability_extending_session(current_session_count)
        
        return all([extending_session >= .3, extending_session <= .7])
        
    
    def _probability_extending_session(self, current_session_count):
        if self.metadata['incentive_index'] == 0:
            return 0
        
        scale = max(5, int(self.metadata['session_size'] / 4))
        continue_session = norm(
            loc=self.metadata['incentive_index'],
            scale=scale
        ).cdf(current_session_count)
        
        return continue_session
        

    def _get_events(self, user_id, session):
        subset = self.dataset[
            (self.dataset['user_id'] == user_id) &
            (self.dataset['session_30_raw'] == session)
        ]
   
        return subset.sort_values(by=['date_time']).reset_index(drop=True)
    
    def _take_action(self, action):
        if action == 0 or self.metadata['incentive_index'] > 0:
            return
        
        current_session_index = min(self.current_session_index, self.current_session.shape[0] - 1)
        self.metadata['incentive_index'] = self.current_session.iloc[current_session_index]['cum_session_event_raw']
        
    def _state(self):

        if self.current_session_index > self.n_sequences:
            events = self.current_session.iloc[self.current_session_index - (self.n_sequences + 1):self.current_session_index][self.out_features].values
            
        else:
            delta = min((self.n_sequences + 1)- self.current_session_index, self.n_sequences)
            zero_cat = np.zeros((delta, len(self.out_features)))
            events = self.current_session.iloc[:max(self.current_session_index, 1)][self.out_features].values
            events = np.concatenate((zero_cat, events), axis=0)
            

        return events.astype(np.float32).T
  
    
    def dists(self):
        metadata_container = self.metadata_container.copy()
        self.metadata_container = []
        return np.array(metadata_container)

In [9]:
# %load policies/cnn_policy
# %load policies/cnn_policy
from typing import Dict, List, Type, Union

import gym
import torch
from gym import spaces
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
from stable_baselines3.dqn.policies import DQNPolicy
from torch import nn


class CustomConv1dFeatures(BaseFeaturesExtractor):
    
    @classmethod
    def setup_sequences_features(cls, n_sequences, n_features):
        cls.n_sequences = n_sequences
        cls.n_features = n_features
        
    
    def __init__(self, observation_space: spaces.Box, features_dim=20):
        super().__init__(observation_space, features_dim)
        
        
        self.cnn_1 = nn.Sequential(
            nn.Conv1d(self.n_features, self.n_features*2, kernel_size=3, padding=1),
            nn.BatchNorm1d(self.n_features*2),
            nn.ReLU(),
            
            nn.Conv1d(self.n_features*2, self.n_features*2, kernel_size=3, padding=1),
            nn.BatchNorm1d(self.n_features*2),
            nn.ReLU(),
            
            nn.Conv1d(self.n_features*2, self.n_features*2, kernel_size=3, padding=1),
            nn.BatchNorm1d(self.n_features*2),
            nn.Conv1d(self.n_features*2, self.n_features*2, kernel_size=3, padding=1),
            
            nn.AvgPool1d(2)
        )
        
        self.cnn_2 = nn.Sequential(
            nn.Conv1d(self.n_features*2, self.n_features, kernel_size=3, padding=1),
            nn.BatchNorm1d(self.n_features),
            nn.ReLU(),
            
            nn.Conv1d(self.n_features, self.n_features, kernel_size=3, padding=1),
            nn.BatchNorm1d(self.n_features),
            nn.ReLU()
        )
        
        self.act = nn.Sequential(
            nn.MaxPool1d(2),
            nn.Flatten(),
        )
        
        with torch.no_grad():
            out_shape = self.act(self.cnn_2(self.cnn_1(torch.zeros((1, self.n_features, self.n_sequences))))).shape[1]
            self.linear = nn.Linear(out_shape, features_dim)
    
    def forward(self, obs):
        out = self.cnn_1(obs)
        out = self.cnn_2(out)
        out = self.act(out)
        return self.linear(out)


        

In [10]:
# %load incentive_reinforcement_learning_cpu.py
import argparse
import logging
import os
from datetime import datetime
from functools import reduce
from pprint import pformat
from typing import Callable
# import boto3
import random
import numpy as np
import pandas as pd
import torch
# from callback import DistributionCallback
# from environment import CitizenScienceEnv
# from policies.cnn_policy import CustomConv1dFeatures
# from rl_constant import LABEL, METADATA, OUT_FEATURE_COLUMNS, PREDICTION_COLS
from stable_baselines3 import A2C, DQN, PPO
from stable_baselines3.common.callbacks import (CallbackList,
                                                CheckpointCallback,
                                                StopTrainingOnMaxEpisodes)
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.vec_env import DummyVecEnv, VecMonitor
from stable_baselines3.dqn.policies import DQNPolicy

ALL_COLS = METADATA + OUT_FEATURE_COLUMNS 

logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %I:%M:%S %p', level=logging.INFO)
np.set_printoptions(precision=4, linewidth=200, suppress=True)
torch.set_printoptions(precision=2, linewidth=200, sci_mode=False)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 500)

S3_BASELINE_PATH = 's3://dissertation-data-dmiller'
USER_INDEX = 1
SESSION_INDEX = 2
CUM_SESSION_EVENT_RAW = 3
TIMESTAMP_INDEX = 11
TRAIN_SPLIT = 0.7
EVAL_SPLIT = 0.15


def train_eval_split(dataset, logger):
    train_split = int(dataset.shape[0] * TRAIN_SPLIT)
    eval_split = int(dataset.shape[0] * EVAL_SPLIT)
    test_split = dataset.shape[0] - train_split - eval_split
    logger.info(f'Train size: 0:{train_split}, eval size: {train_split}:{train_split+eval_split}: test size: {train_split + eval_split}:{dataset.shape[0]}')
    train_dataset, eval_dataset, test_split = dataset[:train_split], dataset[train_split:train_split+eval_split], dataset[train_split+eval_split:]
    
    return {
        'train': train_dataset,
        'eval': eval_dataset,
        'test': test_split
    }

def generate_metadata(dataset):
    
    session_size = dataset.groupby(['user_id', 'session_30_raw'])['size_of_session'].max().reset_index(name='session_size')
    session_minutes = dataset.groupby(['user_id', 'session_30_raw'])['cum_session_time_raw'].max().reset_index(name='session_minutes')
    
    sim_minutes = dataset.groupby(['user_id', 'session_30_raw'])['cum_session_time_raw'].quantile(.7, interpolation='nearest').reset_index(name='sim_minutes')
    sim_size = dataset.groupby(['user_id', 'session_30_raw'])['cum_session_event_raw'].quantile(.7, interpolation='nearest').reset_index(name='sim_size')
    
    
    sessions = [session_size, session_minutes, sim_minutes, sim_size]
    sessions = reduce(lambda left, right: pd.merge(left, right, on=['user_id', 'session_30_raw']), sessions)
    dataset = pd.merge(dataset, sessions, on=['user_id', 'session_30_raw'])
    dataset['reward'] = dataset['cum_session_time_raw']
    return dataset


def parse_args():
    parse = argparse.ArgumentParser()
    parse.add_argument('--read_path', type=str, default='datasets/rl_ready_data')
    parse.add_argument('--n_files', type=int, default=2)
    parse.add_argument('--n_episodes', type=int, default=50)
    parse.add_argument('--n_sequences', type=int, default=40)
    parse.add_argument('--n_envs', type=int, default=100)
    parse.add_argument('--lstm', type=str, default='seq_10')
    parse.add_argument('--device', type=str, default='cpu')
    parse.add_argument('--checkpoint_freq', type=int, default=1000)
    parse.add_argument('--tb_log', type=int, default=100)
    parse.add_argument('--feature_extractor', type=str, default='cnn') 
    parse.add_argument('--window', type=int, default=4)
    args = parse.parse_args()
    return args



def run_reinforcement_learning_incentives(environment, logger, n_episodes=1):
    for epoch in range(n_episodes):
        environment_comp = False
        state = environment.reset()
        i = 0
        while not environment_comp:
            next_action = (
                1 if np.random.uniform(low=0, high=1) > 0.8 else 0
            )
            state, rewards, environment_comp, meta = environment.step(next_action)
            i +=1
            if i % 100 == 0:
                logger.info(f'Step: {i} - Reward: {rewards}')
                
        logger.info(f'Epoch: {epoch} - Reward: {rewards}')
        print(environment.user_sessions.head(10))


def remove_events_in_minute_window(df):
    df['second_window'] = df['second'] // 10
    df = df.drop_duplicates(
        subset=['user_id', 'session_30_raw', 'year', 'month', 'day', 'hour', 'minute'],
        keep='last'
    ).reset_index(drop=True)

    return df


def convolve_delta_events(df, window):
    df['convolved_delta_event'] = (
        df.set_index('date_time').groupby(by=['user_id', 'session_30_raw'], group_keys=False) \
            .rolling(f'{window}T', min_periods=1)['delta_last_event'] \
            .mean()
            .reset_index(name='convolved_event_delta')['convolved_event_delta']
    )

    df['delta_last_event'] = df['convolved_delta_event']

    return df

random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
    
def main(args):
    
    exec_time = datetime.now().strftime("%Y-%m-%d-%H-%M")
    logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %I:%M:%S %p', level=logging.INFO)
    logger = logging.getLogger(__name__)
    logger.setLevel(logging.INFO)
    
    logger.info('Starting Incentive Reinforcement Learning')
    
    read_path, n_files, n_sequences, n_episodes, device, n_envs, lstm, tb_log, check_freq, feature_ext, window = (
        args.read_path, 
        args.n_files, 
        args.n_sequences, 
        args.n_episodes, 
        args.device,
        args.n_envs,
        args.lstm,
        args.tb_log,
        args.checkpoint_freq,
        args.feature_extractor,
        args.window
    )
    
    read_path = os.path.join(
        read_path,
        f'files_used_{n_files}',
        f'predicted_data.parquet'
    )
   
    # if not os.path.exists(read_path):
    #     logger.info(f'Downloading data from {S3_BASELINE_PATH}/{read_path}') 
    #     s3 = boto3.client('s3')
    #     s3.download_file(
    #         S3_BASELINE_PATH,     
    #         read_path,
    #         read_path
    #     )
             
            
    logger.info(f'Reading data from {read_path}')
    
    logger.info(f'Setting up model with prediction {lstm}')
    df = pd.read_parquet(read_path, columns=ALL_COLS + [args.lstm] if args.lstm else ALL_COLS)
    df['date_time'] = pd.to_datetime(df[['year', 'month', 'day', 'hour', 'minute', 'second']], errors='coerce')


    df = df.sort_values(by=['date_time'])    
    
    logger.info(f'N events:: {df.shape[0]} creating training partitions')
    df = df.head(int(df.shape[0] * .7))
    logger.info(f'N events after 70% split: {df.shape[0]}')
    size_of_session = df.groupby(['user_id', 'session_30_raw']).size().reset_index(name='size_of_session')
    df = pd.merge(df, size_of_session, on=['user_id', 'session_30_raw'])
    df['cum_session_event_raw'] = df.groupby(['user_id', 'session_30_raw'])['date_time'].cumcount() + 1
    
    logger.info(f'Convolution over {window} minute window')
    df = convolve_delta_events(df, window)
    logger.info(f'Convolving over {window} minute window complete: generating metadata')
    df = generate_metadata(df) 
    logger.info(f'Metadata generated: selecting events only at {window} minute intervals')
    df = df[df['minute'] % window == 0]
    logger.info(f'Data read: {df.shape[0]} rows, {df.shape[1]} columns, dropping events within 2 minute window')
    df = remove_events_in_minute_window(df)
    df = df.reset_index(drop=True)
    
    logger.info(f'Number of events after dropping events within 2 minute window: {df.shape[0]}')
    
    unique_episodes = df[['user_id', 'session_30_raw']].drop_duplicates()
    unique_sessions = df[['session_30_raw']].drop_duplicates()
    df = df.drop(columns=['year', 'month', 'day', 'hour', 'minute', 'second', 'second_window'])
    out_features = OUT_FEATURE_COLUMNS + [lstm] if lstm else OUT_FEATURE_COLUMNS
    

    
    citizen_science_vec =DummyVecEnv([lambda: CitizenScienceEnv(df, out_features, n_sequences) for i in range(n_envs)])
    logger.info(f'Vectorized environments created')
    
    base_path = os.path.join(
        # S3_BASELINE_PATH,
        'reinforcement_learning_incentives_2',
        f'n_files_{n_files}',
        feature_ext + '_' + 'label' if lstm.startswith('continue') else lstm,
        'results',
        exec_time,
    ) 
    
    
    tensorboard_dir, checkpoint_dir = (
        os.path.join(base_path, 'training_metrics'),
        os.path.join(base_path, 'checkpoints')
    )

    logger.info(f'Creating callbacks, monitors and loggerss')
    callback_max_episodes = StopTrainingOnMaxEpisodes(max_episodes=n_episodes, verbose=1)
    checkpoint_callback = CheckpointCallback(save_freq=check_freq // (n_envs // 2), save_path=checkpoint_dir, name_prefix='rl_model')
    dist_callback = DistributionCallback()
    DistributionCallback.tensorboard_setup(tensorboard_dir, tb_log)
    callback_list = CallbackList([callback_max_episodes, dist_callback, checkpoint_callback])
    monitor_train = VecMonitor(citizen_science_vec)
    
    
    if feature_ext == 'cnn':
        CustomConv1dFeatures.setup_sequences_features(n_sequences + 1, 21)
        logger.info('Using custom 1 dimensional CNN feature extractor')
        policy_kwargs = dict(
            features_extractor_class=CustomConv1dFeatures,
            net_arch=[10]
        )
        model = DQN(policy='CnnPolicy', env=monitor_train, verbose=1, tensorboard_log=tensorboard_dir, policy_kwargs=policy_kwargs, device=device, stats_window_size=1000)
    else:
        logger.info('Using default MLP feature extractor')
        model = DQN(policy='MlpPolicy', env=monitor_train, verbose=1, tensorboard_log=tensorboard_dir, device=device, stats_window_size=1000)
    
    logger.info(f'Model created: policy')
    
    logger.info(pformat(model.policy))
        
    logger.info(f'Beginning training') 
    
    # retutrn
            
    logger.info(pformat([
        'n_episodes: {}'.format(n_episodes),
        'read_path: {}'.format(read_path),
        'n_files: {}'.format(n_files),
        'n_sequences: {}'.format(n_sequences),
        'n_envs: {}'.format(n_envs),
        'total_timesteps: {}'.format(df.shape),
        f'unique_episodes: {unique_episodes.shape[0]}',
        'device: {}'.format(device),
        'tensorboard_dir: {}'.format(tensorboard_dir),
        'checkpoint_dir: {}'.format(checkpoint_dir)
    ]))
    
    model.learn(total_timesteps=15_000_000, progress_bar=True, log_interval=1000, callback=callback_list)
    


In [11]:
class Argument:
    read_path = 'rl_ready_data'
    n_files = 30
    n_sequences = 40
    n_episodes = 500_000
    n_envs = 1000
    lstm = 'seq_40'
    device = 'cuda'
    checkpoint_freq = 250_000
    tb_log = 50
    feature_extractor = 'cnn'
    window = 4

In [12]:

main(Argument)

05/28/2023 11:27:35 AM Starting Incentive Reinforcement Learning
05/28/2023 11:27:35 AM Reading data from rl_ready_data/files_used_30/predicted_data.parquet
05/28/2023 11:27:35 AM Setting up model with prediction seq_40
05/28/2023 11:27:51 AM N events:: 38500990 creating training partitions
05/28/2023 11:27:51 AM N events after 70% split: 26950693
05/28/2023 11:28:08 AM Convolution over 4 minute window
05/28/2023 11:28:27 AM Convolving over 4 minute window complete: generating metadata
05/28/2023 11:28:51 AM Metadata generated: selecting events only at 4 minute intervals
05/28/2023 11:28:57 AM Data read: 6736552 rows, 43 columns, dropping events within 2 minute window
05/28/2023 11:28:59 AM Number of events after dropping events within 2 minute window: 1173560
05/28/2023 11:28:59 AM Vectorized environments created
05/28/2023 11:28:59 AM Creating callbacks, monitors and loggerss
05/28/2023 11:28:59 AM Using custom 1 dimensional CNN feature extractor


Using cuda device


05/28/2023 11:29:01 AM Model created: policy
05/28/2023 11:29:01 AM CnnPolicy(
  (q_net): QNetwork(
    (features_extractor): CustomConv1dFeatures(
      (cnn_1): Sequential(
        (0): Conv1d(21, 42, kernel_size=(3,), stride=(1,), padding=(1,))
        (1): BatchNorm1d(42, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): Conv1d(42, 42, kernel_size=(3,), stride=(1,), padding=(1,))
        (4): BatchNorm1d(42, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): ReLU()
        (6): Conv1d(42, 42, kernel_size=(3,), stride=(1,), padding=(1,))
        (7): BatchNorm1d(42, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (8): Conv1d(42, 42, kernel_size=(3,), stride=(1,), padding=(1,))
        (9): AvgPool1d(kernel_size=(2,), stride=(2,), padding=(0,))
      )
      (cnn_2): Sequential(
        (0): Conv1d(42, 21, kernel_size=(3,), stride=(1,), padding=(1,))
        (1): BatchNorm1d(21, eps=1e-05

Logging to reinforcement_learning_incentives_2/n_files_30/seq_40/results/2023-05-28-11-27/training_metrics/DQN_1


Output()